In [2]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from facenet_pytorch import InceptionResnetV1
from itertools import combinations
import random
from torchvision import transforms
from PIL import Image
import torch
from sklearn.neighbors import KNeighborsClassifier
import numpy as np 

c:\Users\julie\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset_path = "../../../ChimpRec-Dataset/Chimpanzee_recognition_dataset"

transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [4]:
class TripletDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
        self.data = []
        self.labels = []
        for img, label in self.dataset:
            self.data.append(img)
            self.labels.append(label)
        self.labels = torch.tensor(self.labels)
        self.classes = torch.unique(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        anchor_img = self.data[idx]
        anchor_label = self.labels[idx]
        
        #On prend un exemple positif et un négatif par rapport à l'anchor (random pour l'instant)
        positive_indices = torch.where(self.labels == anchor_label)[0]
        negative_indices = torch.where(self.labels != anchor_label)[0]

        positive_idx = random.choice(positive_indices)
        negative_idx = random.choice(negative_indices)

        positive_img = self.data[positive_idx]
        negative_img = self.data[negative_idx]

        return anchor_img, positive_img, negative_img


In [5]:
train_dataset = TripletDataset(
    datasets.ImageFolder(os.path.join(dataset_path, "train"), transform=transform),
    transform=transform
)
val_dataset = TripletDataset(
    datasets.ImageFolder(os.path.join(dataset_path, "val"), transform=transform),
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

In [6]:
facenet = InceptionResnetV1(pretrained='vggface2').eval()

In [7]:
# Débloquer les dernières couches
for param in facenet.parameters():
    param.requires_grad = False
for layer in list(facenet.children())[-5:]:
    for param in layer.parameters():
        param.requires_grad = True

# Déplacer le modèle sur GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
facenet = facenet.to(device)

In [8]:
criterion = nn.TripletMarginLoss(margin=1.0)
optimizer = optim.Adam(facenet.parameters(), lr=0.001)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        print("-----------------------")

        model.train()
        running_loss = 0.0

        for anchor, positive, negative in train_loader:
            anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

            optimizer.zero_grad()

            anchor_output = model(anchor)
            positive_output = model(positive)
            negative_output = model(negative)

            loss = criterion(anchor_output, positive_output, negative_output)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f"Train Loss: {epoch_loss:.4f}")

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for anchor, positive, negative in val_loader:
                anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

                anchor_output = model(anchor)
                positive_output = model(positive)
                negative_output = model(negative)

                loss = criterion(anchor_output, positive_output, negative_output)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        print(f"Val Loss: {val_loss:.4f}")

train_model(facenet, train_loader, val_loader, criterion, optimizer, num_epochs=10)



Epoch 1/10
-----------------------


Pour prédire:

In [ ]:
def training_embedding_label(model, train_loader, device, k=5):
    model.eval()
    embeddings = []
    labels = []

    with torch.no_grad():
        for inputs, targets in train_loader.dataset.dataset:
            inputs = inputs.to(device).unsqueeze(0)
            embedding = model(inputs).cpu().squeeze().numpy()
            embeddings.append(embedding)
            labels.append(targets)

    embeddings = np.array(embeddings)
    labels = np.array(labels)

    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(embeddings, labels)

    print(f"Training embeddings and k-NN completed. k-NN trained with {len(labels)} samples.")
    return knn, embeddings, labels


In [ ]:
def predict(model, knn, image_path, transform, device):
    image = Image.open(image_path).convert("RGB")
    
    image = transform(image).unsqueeze(0)  

    model.eval()
    with torch.no_grad():
        image = image.to(device)
        embedding = model(image).cpu().squeeze().numpy()

    predicted_class = knn.predict([embedding])[0]

    return predicted_class


In [ ]:
knn, train_embeddings, train_labels = training_embedding_label(facenet, train_loader, device, k=5)

image_path = "../../../ChimpRec-Dataset/Chimpanzee_recognition_dataset/val/JEJE/JEJE_8.jpg"
predicted_class = predict(facenet, knn, image_path, transform, device)

print(f"Predicted Class: {predicted_class}")


Avec la préçision de la classifiactio grâçe à un k-NN classificateur:

In [ ]:
"""def train_model_with_knn(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, k=5):
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        print("-----------------------")

        # Phase d'entraînement
        model.train()
        running_loss = 0.0

        for anchor, positive, negative in train_loader:
            anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

            optimizer.zero_grad()

            anchor_output = model(anchor)
            positive_output = model(positive)
            negative_output = model(negative)

            loss = criterion(anchor_output, positive_output, negative_output)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f"Train Loss: {epoch_loss:.4f}")

        # Phase de validation avec Triplet Loss
        model.eval()
        val_loss = 0.0
        embeddings = []
        labels = []

        with torch.no_grad():
            for anchor, positive, negative in val_loader:
                anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

                anchor_output = model(anchor)
                positive_output = model(positive)
                negative_output = model(negative)

                loss = criterion(anchor_output, positive_output, negative_output)
                val_loss += loss.item()

        

train_model(facenet, train_loader, val_loader, criterion, optimizer, num_epochs=10, k=10)"""